In [1]:
# This notebook is going to use the data-dashboard code to add SCHEDULED_HEADWAYS from gtfs to events (ARR/DEP) data.
# It's complicated and I don't want to re-write the logic, so I"m going to use it as is.
# Thus, we import bus2train and call it on our data.

import bus2train
import pandas as pd
import pathlib
import glob
import numpy as np

In [48]:
input_file = "./data/in-data/2023/MBTA-Bus-Arrival-Departure-Times_2023-08.csv"
output_dir = "./data/out-data/"

raw_dat = pd.read_csv(input_file)
all_routes = [r.lstrip("0") for r in raw_dat.route_id.value_counts().index]

In [49]:
# this will take a few minutes to run since it has a lot of data to process
# and a lot of files to write to disk
# It will start by downloading a bunch of GTFS feeds, then calculate and add the scheduled_headways 
# to the events, and save the output in output_dir as gzipped csv.

pathlib.Path(output_dir).mkdir(exist_ok=True)

data = bus2train.load_data(input_file, all_routes)
events = bus2train.process_events(data)
bus2train.to_disk(events, output_dir, nozip=False) 
# set nozip to True if you want to be able to read the files. The data is large though... 59M compressed.
# Maybe run on a subset of routes if you want to experiment with unzipped.

Archive for 20230802 already exists: 20230728
Archive for 20230803 already exists: 20230728
Archive for 20230805 already exists: 20230804
Archive for 20230806 already exists: 20230804
Archive for 20230807 already exists: 20230804
Archive for 20230808 already exists: 20230804
Archive for 20230809 already exists: 20230804
Archive for 20230811 already exists: 20230810
Archive for 20230812 already exists: 20230810
Archive for 20230813 already exists: 20230810
Archive for 20230814 already exists: 20230810
Archive for 20230816 already exists: 20230815
Archive for 20230818 already exists: 20230817
Archive for 20230820 already exists: 20230819
Archive for 20230821 already exists: 20230819
Archive for 20230823 already exists: 20230822
Archive for 20230826 already exists: 20230825
Archive for 20230827 already exists: 20230825
Archive for 20230828 already exists: 20230825
Archive for 20230829 already exists: 20230825


In [38]:
# While it is possible to load all this data into a single dataframe,
# the volume means its easier to work with if it's broken into chunks
# That you can then read/load/process individually.

In [59]:
# routes = ["111", "39"]
MONTH=10
routes = all_routes

direction = "*" # use 0 and 1 if you want to specify outbound and inbound. * will use both.

OUTPUT = dict()

for ROUTE in routes:
    files = glob.glob(f"{output_dir}/Events/monthly-bus-data/{ROUTE}-{direction}-*/**/Month={MONTH}/events.csv.gz", recursive=True)

    route_output = []
    for f in files:
        df = pd.read_csv(f, parse_dates=['service_date',"event_time"])
        df = df[(df.event_type == "ARR") & (df.service_date.dt.dayofweek < 5) & df.event_time.dt.hour.isin(range(7,19))]
        df['actual_headway'] = df.groupby('service_date').event_time.diff().dt.seconds

        df = df[['actual_headway', 'scheduled_headway']].dropna()

        for n in [0.1, 0.2, 0.3, 0.4, 0.5, 0.25]:
            df['bunched_' + str(n)] = df.actual_headway / df.scheduled_headway < n
        
        route_output.append(df)

    output = pd.concat(route_output)
    n_trips = len(output)
    output = output.agg("mean")
    output.rename({"actual_headway": "mean_actual_headway", "scheduled_headway": "mean_scheduled_headway"}, inplace=True)
    output["trips"] = n_trips
    output["route_id"] = ROUTE

    OUTPUT[ROUTE] = output

        

In [60]:
data = pd.concat(OUTPUT.values(), axis=1).T.set_index("route_id")
# data.to_csv("bunchie_data_breakdown.csv")

In [65]:
data[data.trips > 4000].sort_values(by='bunched_0.25', ascending=False).head(20) # oct

,mean_actual_headway,mean_scheduled_headway,bunched_0.1,bunched_0.2,bunched_0.3,bunched_0.4,bunched_0.5,bunched_0.25,trips
route_id,,,,,,,,,
1,785.64386,680.134612,0.084053,0.156606,0.200223,0.240076,0.280036,0.176533,18869.0
66,699.536885,623.262925,0.079153,0.131214,0.179248,0.21403,0.257994,0.160794,22109.0
111,393.758025,361.980043,0.090286,0.136872,0.187876,0.237858,0.27726,0.160525,33272.0
22,732.328981,642.540402,0.078488,0.128301,0.176143,0.213835,0.253203,0.155302,20296.0
57,686.967122,613.007547,0.069555,0.121537,0.157475,0.196264,0.234313,0.138741,18949.0
23,648.068407,561.73188,0.063675,0.113352,0.156862,0.195913,0.238453,0.137976,25787.0
28,736.453133,630.022649,0.059681,0.110105,0.150157,0.192196,0.234752,0.132475,25167.0
16,1053.005076,800.214648,0.056635,0.103988,0.140827,0.176287,0.211168,0.123495,13790.0
70,1034.251547,939.046738,0.05435,0.098029,0.129829,0.16177,0.196841,0.116952,14057.0


In [54]:
data[data.trips > 100].sort_values(by='bunched_0.25', ascending=False).head(20) #oct
# TODO: check for nan timepoints adding to the bunch (for the 442 esp.)

,mean_actual_headway,mean_scheduled_headway,bunched_0.1,bunched_0.2,bunched_0.3,bunched_0.4,bunched_0.5,bunched_0.25,trips
route_id,,,,,,,,,
1,811.936822,678.465894,0.072249,0.137052,0.17606,0.215645,0.249096,0.154616,19073.0
57,673.132097,573.317507,0.069486,0.123244,0.165875,0.201137,0.24273,0.147626,20220.0
66,729.153609,630.594933,0.068126,0.114834,0.158089,0.192012,0.233913,0.140171,21431.0
111,412.311734,355.453009,0.074153,0.112603,0.159262,0.201334,0.23909,0.133066,33134.0
442,2416.652654,3382.910267,0.021288,0.081919,0.175963,0.254918,0.331717,0.129884,3711.0
22,756.216512,660.496834,0.058402,0.099318,0.138773,0.170093,0.203751,0.122211,20530.0
28,757.0606,632.986304,0.049323,0.090373,0.123581,0.1555,0.191634,0.108284,25627.0
23,670.442762,568.819966,0.045699,0.085378,0.120649,0.152162,0.18912,0.104854,26084.0
39,759.270005,660.898187,0.039462,0.076726,0.108726,0.142419,0.179271,0.09403,21844.0


In [58]:
data[data.trips > 100][["bunched_0.25"]].sort_values(by='bunched_0.25', ascending=False).head(20) # aug


,bunched_0.25
route_id,
1,0.154616
57,0.147626
66,0.140171
111,0.133066
442,0.129884
22,0.122211
28,0.108284
23,0.104854
39,0.09403


In [47]:
data.loc["39"]

actual_headway       806.298196
scheduled_headway     660.70002
bunched                0.075199
trips                   19628.0
Name: 39, dtype: object

In [48]:
data.sort_values(by='trips', ascending=False).head(10)

,actual_headway,scheduled_headway,bunched,trips
route_id,,,,
111,396.506665,320.206604,0.163708,33010.0
28,731.504401,577.676126,0.136046,25337.0
23,682.475378,534.82454,0.113742,24450.0
66,724.455027,576.724817,0.143829,21324.0
57,640.619771,469.360103,0.149046,20222.0
1,745.209439,553.333333,0.165883,19791.0
39,806.298196,660.70002,0.075199,19628.0
22,831.935275,633.520778,0.117303,17783.0
743,896.567702,659.767984,0.062607,16292.0
